In [1]:
from scipy.optimize import root
import numpy as np

/rhome/yyang440/bigdata/anaconda3/envs/nbodykit-env/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
np.log(3)

1.0986122886681098

In [11]:
# error function parameters
rho = 5.53945717
eta = 0.12420984
beta_L = 0.63021449
beta_H = 2.17635687

# computational cost
C_L = 1.97  # C_L1 + C_L2 in CPU hours
C_H = 230.59

# objective error
phi_0 = .01


In [6]:
# solving for optimal n_LF and n_HF

# solve for n_LF
def equation_to_solve(x):
    # return .54*x**1.37 + (.787-.023*np.log(x))*x - 2.765
    return .54*x**1.37 + (.684-.0416*np.log(x))*x - 4.416

result = root(equation_to_solve, x0=1.0)
if result.success:
    root_value = result.x
    print("Root:", root_value)
else:
    print("Root finding did not converge.")

# n_LF = root_value

# # solve for n_HF
# n_HF = ((C_L * beta_H)/(rho * C_H * beta_L))**(1/(1 + beta_H)) * n_LF**((1 + beta_L)/(1 + beta_H))


Root: [3.04013076]


In [13]:
# results
print("n_LF:", n_LF)
print("n_HF:", n_HF)

n_LF: [1120.44384067]
n_HF: [7.06744302]


In [14]:
# error function at integer n_LF and n_HF
n_LF = int(n_LF[0])
n_HF = int(n_HF[0])
phi = eta * (rho * n_LF**(-beta_L) + n_HF**(-beta_H))
print("phi at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), "%.2f" % (phi * 100) + "%")

# computational cost at integer n_LF and n_HF
cpus = 56 # frontera
C = C_L * n_LF + C_H * n_HF
print("C at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), C, "CPU hours", "= %.2f" % (C/cpus), "node hours")

phi at n_LF = 1120, n_HF = 7: 1.00%
C at n_LF = 1120, n_HF = 7: 3820.53 CPU hours = 68.22 node hours


In [15]:
# error function at integer n_LF and n_HF
n_LF = 1120
n_HF = 8
phi = eta * (rho * n_LF**(-beta_L) + n_HF**(-beta_H))
print("phi at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), "%.2f" % (phi * 100) + "%")

# computational cost at integer n_LF and n_HF
cpus = 56 # frontera
C = C_L * n_LF + C_H * n_HF
print("C at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), C, "CPU hours", "= %.2f" % (C/cpus), "node hours")

phi at n_LF = 1120, n_HF = 8: 0.96%
C at n_LF = 1120, n_HF = 8: 4051.12 CPU hours = 72.34 node hours
